In [ ]:
from PIL import Image 
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import tensorflow as tf 


# load the data

In [ ]:
train_data = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
test_data = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')

In [ ]:
print(train_data.img_width.unique())
print(train_data.img_height.unique())


In [ ]:
train_images = tf.data.Dataset.list_files('../input/hubmap-organ-segmentation/train_images/*.tiff')
test_images = tf.data.Dataset.list_files('../input/hubmap-organ-segmentation/test_images/*.tiff')

In [ ]:
# get the rle of the training data 
train_rle_arr= np.array(train_data.rle) 

In [ ]:
train_rle = tf.data.Dataset.from_tensor_slices(train_rle_arr)

In [ ]:
dataset = tf.data.Dataset.zip((train_images , train_rle))

In [ ]:
# convert rle to mask 
def interpret_rle(rle , width , target_size ):
    rle_arr= np.array(rle.split() , dtype = int)
    # get start of each annotation 
    start = rle_arr[::2] 
    steps = rle_arr[1::2] 
    end = start + steps
    width , width = width
    img = np.zeros(shape = (width * width ))
    for st , en in zip(start ,end) : 
        img[st : en] = 1 
    
    img = Image.fromarray(img.reshape(width, width))
    
    img = img.resize((target_size, target_size))
    img = np.array(img).astype(float)
    #rescale label
    img = np.round((img - img.min())/(img.max() - img.min()))
    
    return img.T

# preprocess the dataset


In [ ]:

def preprocess(train_image_path , rle) : 
    # load the train images 
    train_img = Image.open(train_image_path.numpy()) 
    # get the mask 
    mask_img = interpret_rle(rle.numpy() , train_img.size  , 128 )
    mask_img = np.expand_dims(mask_img , -1)
    # resize and rescale the images 
    train_img = train_img.resize((128 , 128 )) 
    train_img = np.array(train_img )
    train_img = train_img / 255. 
    mask_img = mask_img / 255. 
    return train_img , mask_img 

# data augmention


In [ ]:
# augment the data with fixed seed to get the same changes for both the image and mask 
class Augment(tf.keras.layers.Layer) : 
    def __init__(self , seed = 42) : 
        super().__init__()
        self.flip_inputs = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=seed)
        self.flip_labels = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=seed)
        self.rotate_inputs = tf.keras.layers.RandomRotation(.2 , seed=seed)
        self.rotate_labels = tf.keras.layers.RandomRotation(.2 , seed = seed)
    def call(self , inputs , labels)  :
        X = self.flip_inputs(inputs)
        y = self.flip_labels(labels)
        X = self.rotate_inputs(X)
        y = self.rotate_labels(y)
        return X  , y 
        

In [ ]:
dataset = dataset.map(lambda x , y : tf.py_function(preprocess, [x , y], [tf.float32 , tf.float32]))

In [ ]:
dataset = dataset.cache().shuffle(200).batch(8).map(Augment()).prefetch(4)

In [ ]:
# split the data 
train_dataset = dataset.take(38)
val_dataset  = dataset.skip(38)

# the model 

## the U-net model

In [ ]:
def conv_block(X , n_filters , pooling = True ) : 
    X = tf.keras.layers.Conv2D(n_filters , (3,3) , padding='same' , activation ='relu')(X)
    X =  tf.keras.layers.Conv2D(n_filters , (3,3) , padding='same' , activation ='relu')(X)
    if pooling : 
        P = tf.keras.layers.MaxPooling2D((2,2))(X) 
        P = tf.keras.layers.Dropout(.3)(P)
        return P , X
    return X 
def conv_transpose_block(X , skip_connection , n_filters ): 
    X_t = tf.keras.layers.Conv2DTranspose(n_filters , (3,3) , strides=(2,2) , activation='relu' , padding ='same')(X)
    X_t = tf.keras.layers.Concatenate()([X_t , skip_connection])
    return conv_block(X_t , n_filters , pooling = False ) 

def Model(input_shape) : 
    X = tf.keras.layers.Input(input_shape)
    P , skip_1 = conv_block(X , 64)
    P , skip_2 = conv_block(P , 128)
    P , skip_3 = conv_block(P , 256)
    P , skip_4 = conv_block(P , 512)
    
    bottle_nick  = conv_block(P , 1024 , pooling = False )
    
    T = conv_transpose_block(bottle_nick , skip_4 , 512)
    T = conv_transpose_block(T , skip_3 , 256)
    T = conv_transpose_block(T , skip_2 , 128)
    T = conv_transpose_block(T , skip_1 , 64)
    
    out = tf.keras.layers.Conv2D(2, (1,1) , padding ='same' , activation='softmax')(T) 
    
    return tf.keras.Model(X  , out )
    
    
    

In [ ]:
from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    # flatten
    y_true_f = K.flatten(K.cast(y_true, np.float32))
    #y_pred_f = K.flatten(K.cast(K.argmax(y_pred, axis=-1), np.float32))
    y_pred_f = K.flatten(K.cast(K.argmax(y_pred, axis=-1), np.float32))
    # compute numerator and denominator
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f)
    # apply dice formula
    dice = K.mean((2.*intersection + smooth)/(union + smooth))
    return dice

In [ ]:
model = Model((128 ,128 , 3))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy' , dice_coef])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='TB')

In [ ]:
EPOCHS = 20

model_history = model.fit(train_dataset, epochs=EPOCHS,
                         validation_data=val_dataset )